# Support Vector Machine Predictive Model for EDTA QC Evaluation: Part 2 – Model Optimization

We will build a **Support Vector Machine** (SVM) classifier using **scikit-learn**. Two datasets (training and unknown) are used for modeling. The training dataset contains experimental data from healthy donors (described in the current manuscript: https://........). The unknown dataset contains experimental samples from healthy donors and colorectal carcinoma donors (Slyskova et al. 2015: https://pubmed.ncbi.nlm.nih.gov/24585457/). The SVM will predict blood sample quality from EDTA tubes (normal or altered gene expression) for downstream gene expression analysis.

Contents:
- **Module Imports**

- **Prediction on Unseen Data and Optimalization of the Decision Boundary**
    - Importing data
    - Importing joblib libraries
    - Extracting data and pre-processing them
    - Predictions
    - Adjusting the decision threshold
    - Evaluation and visualization of the decision boundary
    - Key hyperparameters
    - Analyzing threshold ranges without changing FNR and FPR
    - Determining FNR and threshold ranges for reliable predictions
    - Selecting the decision threshold based on FNR
    - Built-in data and final outcome from EDTA_QC testing


----
----

#### Module Imports

In [7]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import pathlib
from pathlib import Path

In [16]:
# ── Setup paths relative to project root ──
cwd = Path().cwd()
ROOT = cwd
while not (ROOT / "README.md").exists():
    ROOT = ROOT.parent
  
DATA_DIR   = ROOT / "data"
MODELS_DIR = ROOT / "models"

MODEL_VERSION = "v8_2025-11-27"
TRAIN_DATA_PATH   = DATA_DIR / "train_165_data.csv"
TEST1_DATA_PATH    = DATA_DIR / "test1_data.csv"
TEST2_DATA_PATH    = DATA_DIR / "test2_data.csv"
UNKNOWN_DATA_PATH    = DATA_DIR / "unknown_only.csv"

PIPELINE_PATH = MODELS_DIR / "final_pipeline_prob_v8_2025-11-27.joblib"
# ── End setup ──

# ── Notebook outputs folder ──
NB_OUTPUT = ROOT / "notebooks" / "outputs_from_notebooks"
NB_OUTPUT.mkdir(parents=True, exist_ok=True)
# ── End notebook outputs setup ──

In [9]:
paths = {
    "train_165": TRAIN_DATA_PATH,
    "test1": TEST1_DATA_PATH,
    "test2": TEST2_DATA_PATH,
    "unknown": UNKNOWN_DATA_PATH
}

----
----

#### **Optimalization of the Decision Boundary and Prediction on Unseen Data**

#### Task 1: Importing data

In [10]:
dfs = {}
samples = {}

for key, path in paths.items():
    data = pd.read_csv(path, index_col=0)
    if "sample" in data.columns:
        samples[key] = data["sample"]
        dfs[key] = data.drop(columns="sample")
    else:
        samples[key] = data.index.astype(str)
        dfs[key] = data


In [11]:
train_165_df   = dfs["train_165"]
test1_df   = dfs["test1"]
test2_df   = dfs["test2"]
unknown_df= dfs["unknown"]

----

#### Task 2: Importing joblib libraries

In [12]:
pipeline = joblib.load(PIPELINE_PATH)

scaler = pipeline.named_steps['scaler']
pca    = pipeline.named_steps['pca']
model  = pipeline.named_steps['model']

----

#### Task 3: Extracting data and pre-processing them

In [13]:
# --- Extract feature columns used in training ---
feature_cols = scaler.feature_names_in_

# --- Split datasets into features (X) and target (y) ---
X_train   = train_165_df[feature_cols]
y_train   = train_165_df["groups"].astype(int)

X_test1   = test1_df[feature_cols]
y_test1   = test1_df["groups"].astype(int)

X_test2   = test2_df[feature_cols]
y_test2   = test2_df["groups"].astype(int)

X_unknown = unknown_df[feature_cols]

In [14]:
X_train_scaled    = scaler.transform(X_train)
X_test1_scaled    = scaler.transform(X_test1)
X_test2_scaled    = scaler.transform(X_test2)
X_unknown_scaled  = scaler.transform(X_unknown)

X_train_pca    = pca.transform(X_train_scaled)
X_test1_pca    = pca.transform(X_test1_scaled)
X_test2_pca    = pca.transform(X_test2_scaled)
X_unknown_pca  = pca.transform(X_unknown_scaled)


----

#### Task 4: Predictions

In [17]:
# --- Predict class probabilities ---
proba_test1   = pipeline.predict_proba(X_test1)
proba_test2   = pipeline.predict_proba(X_test2)
proba_unknown = pipeline.predict_proba(X_unknown)

# --- Predict class labels ---
y_test1_pred   = pipeline.predict(X_test1)
y_test2_pred   = pipeline.predict(X_test2)
y_unknown_pred = pipeline.predict(X_unknown)


AttributeError: This 'Pipeline' has no attribute 'predict_proba'

----

#### Task 5: Adjusting the decision threshold

In [ ]:
# --- Decision scores (distance to boundary) ---
scores_train   = pipeline.decision_function(X_train)
scores_test1   = pipeline.decision_function(X_test1)
scores_test2   = pipeline.decision_function(X_test2)
scores_unknown = pipeline.decision_function(X_unknown)

In [ ]:
# Generating initial predictions with default threshold
y_unk_orig   = (scores_unknown > 0).astype(int)
y_train_orig   = (scores_train > 0).astype(int)
y_test2_orig  = (scores_test2  > 0).astype(int)
y_test1_orig  = (scores_test1  > 0).astype(int)

----

#### Task 6: Evaluation and visualization of the decision boundary

In [ ]:
# Generate meshgrid and decision surface
plt.figure(figsize=(10,8))
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca [:, 0].max() + 1
y_min, y_max = X_train_pca [:, 1].min() - 1, X_train_pca [:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
grid = np.c_[xx.ravel(), yy.ravel()]
Z = model.decision_function(grid).reshape(xx.shape)

# Define custom colormaps
cmap_train  = mcolors.ListedColormap(['#fb9a99', '#a6cee3'])  # Light colors for train data
cmap_unknown =  mcolors.ListedColormap(['#e31a1c', '#1f78b4'])

plt.contourf(xx, yy, Z, alpha=0.2, cmap=plt.cm.coolwarm_r)# Decision boundary
plt.contour(xx, yy, Z, levels=[0], colors='k', linewidths=1) #levels are responsible for the threshold

# Scatter plot: training data
plt.scatter(X_train_pca [:, 0], X_train_pca [:, 1],
    c=y_train, cmap=cmap_train, edgecolors='k', s=80, label='Train')

# Scatter plot: test2 data
plt.scatter(X_test2_pca [:, 0], X_test2_pca [:, 1],
    c=y_test2, cmap=cmap_train , marker="D", edgecolors='k', s=80, label='Test 2')

# Scatter plot: unknown data
plt.scatter(X_unknown_pca[:, 0], X_unknown_pca[:, 1],
    c=y_unknown_pred, cmap=cmap_unknown, edgecolors='k', s=80, label='Unknown')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Polynomial SVM Decision Boundary at Threshold = 0')

# Create legend
patch0_train = mpatches.Patch(color='#a6cee3', label='Training: good mRNA quality')
patch1_train = mpatches.Patch(color='#fb9a99', label='Training: poor mRNA quality')
patch0_test = mpatches.Patch(color='#a6cee3', label='Test 2: good mRNA quality')
patch1_test = mpatches.Patch(color='#fb9a99', label='Test 2: poor mRNA quality')
patch0_unknown = mpatches.Patch(color='#1f78b4', label='Unknowns: good mRNA quality')
patch1_unknown = mpatches.Patch(color='#e31a1c', label='Unknowns: poor mRNA quality')

plt.legend(handles=[patch0_train, patch1_train, patch0_test, patch1_test, patch0_unknown, patch1_unknown], loc='lower left')

plt.show()

----

#### Task 7: Key hyperparameters

In [ ]:
# Key hyperparameters
print("\n=== Key hyperparameters ===")
print(f"Kernel: {model.kernel}")
print(f"Degree: {model.degree}")
print(f"C:      {model.C}")
print(f"Gamma:  {model.gamma}")
print(f"Coef0:  {model.coef0}")
print(f"Total support vectors: {model.support_vectors_.shape[0]}")

print("\n=== Threshold-dependent parameters ===")
print(f"Original threshold: 0")
pos_orig = np.sum(y_unk_orig == 1)
neg_orig = np.sum(y_unk_orig == 0)
print(f"\nNumber of samples classified as OK (1) at threshold 0:  {pos_orig}")
print(f"Number of samples classified as altered (0) at threshold 0: {neg_orig}")


----

#### Task 8: Analyzing threshold ranges without changing FNR and FPR

In [ ]:
# Test data score range
max_neg_score_test2 = scores_test2[y_test2 == 0].max()
min_pos_score_test2 = scores_test2[y_test2 == 1].min()

# Threshold range without altering test predictions, FNR, or FPR
print(f"↔️ Threshold range preserving test predictions, FNR, and FPR "
      f"({max_neg_score_test2:.2f}, {min_pos_score_test2:.2f}) ")


----

#### Task 9: Determining FNR and threshold ranges for reliable predictions

In [ ]:
# FNR values to test
fnr_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
for fnr in fnr_list:
    print(f"\n{'='*40}\nFNR: {fnr}%")

    test_samples = samples["test2"]
    train_samples = samples["train_165"]
    unknown_samples = samples["unknown"]

    # Calculating threshold from test scores for poor-quality samples
    bad_scores_test2 = scores_test2[y_test2 == 0]
    threshold_dynamic2 = np.percentile(bad_scores_test2, 100 - fnr)
    print(f"Calculated threshold based on test data for {fnr}% FNR: {threshold_dynamic2:.2f}")

    # Updating predictions based on dynamic threshold
    y_train_adj = (scores_train > threshold_dynamic2).astype(int)
    y_test1_adj = (scores_test1 > threshold_dynamic2).astype(int)
    y_test2_adj = (scores_test2 > threshold_dynamic2).astype(int)
    y_unk_adj = (scores_unknown > threshold_dynamic2).astype(int)

    plt.hist(scores_test2[y_test2 == 0], bins=20, alpha=0.5, label="Poor samples", color="#fb9a99")
    plt.hist(scores_test2[y_test2 == 1], bins=20, alpha=0.5, label="Good samples", color="#a6cee3")
    plt.axvline(x=0, color='k', linestyle='--', label='Original threshold')
    plt.axvline(x=threshold_dynamic2, color='r', linestyle='--', label='Adjusted threshold')
    plt.legend()
    plt.title("Decision scores distribution — Test set 2")
    plt.xlabel("Decision score")
    plt.ylabel("Count")
    plt.show()

    #----------------------------------------------------------------------------------------------------------------
    # Plotting decision boundary at adjusted threshold
    plt.figure(figsize=(10, 8))
    plt.title(f"SVM Threshold for Decision Boundary at FNR {fnr}% (Threshold={threshold_dynamic2:.2f})")
    x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
    y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.decision_function(grid).reshape(xx.shape)

    #  Define custom colormaps for plotting
    cmap_train = mcolors.ListedColormap(['#fb9a99', '#a6cee3'])  # Light colors for train data
    cmap_unknown =  mcolors.ListedColormap(['#e31a1c', '#1f78b4'])

    plt.contourf(xx, yy, Z, alpha=0.2, cmap=plt.cm.coolwarm_r) # Decision boundary
    plt.contour(xx, yy, Z, levels=[threshold_dynamic2], colors='k', linewidths=1) #levels are responsible for the threshold

    # Scatter plot: training data
    plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1],
        c=y_train, cmap=cmap_train, marker='x', s=80, label='Train')

     # Scatter plot: training data
    plt.scatter(X_test2_pca[:, 0], X_test2_pca[:, 1],
        c=y_test2_adj, cmap=cmap_train, marker='D', s=80, label='Test2')
   
    # Scatter plot: unknown data
    plt.scatter(X_unknown_pca[:, 0], X_unknown_pca[:, 1],
        c=y_unk_adj, cmap=cmap_unknown, edgecolors='k', s=80, label='Unknowns')

    plt.xlabel('PC1')
    plt.ylabel('PC2')

    # Legend patches
    patch0_train = mpatches.Patch(color='#a6cee3', label='Training: good mRNA quality')
    patch1_train = mpatches.Patch(color='#fb9a99', label='Training: poor mRNA quality')
    patch0_test = mpatches.Patch(color='#a6cee3', label='Test 2 (diamonts): good mRNA quality')
    patch1_test = mpatches.Patch(color='#fb9a99', label='Test 2 (diamonts): poor mRNA quality')
    patch0_unknown = mpatches.Patch(color='#1f78b4', label='Unknowns: good mRNA quality')
    patch1_unknown = mpatches.Patch(color='#e31a1c', label='Unknowns: poor mRNA quality')

    plt.legend(handles=[patch0_train, patch1_train, patch0_test, patch1_test, patch0_unknown, patch1_unknown], loc='lower left')
    plt.show()

    # ----------------------------------------------------------------------------------------------------------------------
    # UNKNOWN data (we are interested in)
    good_unk_idx = np.where(y_unk_adj == 1)[0]
    bad_unk_idx = np.where(y_unk_adj == 0)[0]
    changed_unk_idx = np.where(y_unk_adj != y_unk_orig)[0]

    # UNKNOWN data
    if len(changed_unk_idx) > 0:
        if isinstance(unknown_samples, pd.Series):
            changed_unk_samples = unknown_samples.iloc[changed_unk_idx].tolist()
        else:
            changed_unk_samples = unknown_samples[changed_unk_idx].tolist()    

    print("-"*40)
    print("\n[Unknown Data]")
    print(f"Number of samples classified as OK (1): {len(good_unk_idx)}")
    print(f"Number of samples classified as altered (0): {len(bad_unk_idx)}")
    print(f"Number of samples with changed classification compared to the threshold 0: {len(changed_unk_idx)}")

    # Save predictions to CSV in the notebook’s output folder
    output_df = pd.DataFrame({
        'sample': unknown_samples,
        f'predicted_class_FNR{fnr}': y_unk_adj
    })
    filename = NB_OUTPUT / f"unknown_predictions_FNR{fnr}.csv"
    output_df.to_csv(filename, index=False)
    print(f"✅ Unknown data exported to: {filename}")

    
    #-----------------------------------------------------------------------------------------------------------------------
    # TRAIN data (change control)
    print("-"*40)
    print("\n[Original Train Data - Change Control]")
    changed_train_idx = np.where(y_train_adj != y_train_orig)[0]
    if len(changed_train_idx) > 0:
        print(f"⚠️ {len(changed_train_idx)} training samples changed classification compared to original settings!")
        changed_train_samples = train_samples[changed_train_idx].tolist()
        print("Changed samples:")
        print(", ".join(map(str, changed_train_samples)))
    else:
        print("No classification changes compared to original settings.")
    
   
    #-----------------------------------------------------------------------------------------------------------------------
     # TEST data
    good_test_idx = np.where(y_test2_adj == 1)[0]
    bad_test_idx = np.where(y_test2_adj == 0)[0]
    changed_test_idx = np.where(y_test2_adj != y_test2_orig)[0]

    if isinstance(test_samples, pd.Series):
        good_test_samples = test_samples.iloc[good_test_idx].tolist()
        bad_test_samples = test_samples.iloc[bad_test_idx].tolist()
        changed_test_samples = test_samples.iloc[changed_test_idx].tolist()
    else:
        good_test_samples = test_samples[good_test_idx].tolist()
        bad_test_samples = test_samples[bad_test_idx].tolist()
        changed_test_samples = test_samples[changed_test_idx].tolist()

    print("-"*40)
    print("\n[Test 2 Data]")
    print(f"Number of samples classified as OK (1): {len(good_test_idx)}")
    print(", ".join(map(str, good_test_samples)) if good_test_samples else "None samples")
    
    print(f"\nNumber of samples classified as altered (0): {len(bad_test_idx)}")
    print(", ".join(map(str, bad_test_samples)) if bad_test_samples else "None samples")

    print(f"\nNumber of samples with changed classification compared to the threshold 0: {len(changed_test_idx)}")
    if changed_test_samples:
        print(", ".join(map(str, changed_test_samples)))

   

----

#### Task 10: Selecting the decision threshold based on FNR

ℹ️ Rationale for Adjusting the False Negative Rate (FNR) in the Application

Adjusting the FNR shifts the decision threshold toward the OK class, allowing a small fraction of samples with borderline altered gene expression to be classified as good quality samples. Because our altered-expression criteria are very stringent, these marginal cases pose minimal risk.

FNR = 0%**: The strictest setting. The decision threshold sits at the extreme edge of the altered distribution, so all other samples, even borderline cases, are classified as OK.

Maximum FNR = 9%: The highest FNR that still maintains 100% specificity on the training set (i.e., no altered training sample is misclassified).



Our unknown samples were of good quality and were mixed with test samples of poorer quality. Therefore, our hypothesis is to maximize the number of “good” samples correctly identified while ensuring “poor” samples are accurately detected. Based on this, the optimal FNR threshold for our unknown samples was set to 5% (build-in example).



----

#### Task 11: Built-in data and final outcome from EDTA_QC testing

We all know that even the best-intentioned EDTA blood draws can sit too long in the tube and quietly distort mRNA levels. Described EDTA QC tool lets you catch those hidden QC failures before they sneak into your biological analyses. To illustrate its use, we applied our EDTA QC application to an already published dataset. Our built-in example uses the data from Slyskova et al. (2015).

In that study, mRNA expression was measured in four cohorts: healthy volunteers, pre-operative colorectal carcinoma patients, early post-operative patients, and six-month survivors—using a rigorously validated cancer-marker panel. Those markers successfully grouped most samples into two clusters (diseased vs. recovered/healthy), but a few “strayed” into the wrong cluster. Unfortunately, those outliers all had sufficiently high RIN scores, so standard RNA-integrity filters provided no reason to exclude them. That’s where our application comes in.

We developed an EDTA QC marker panel capable of distinguishing “good” quality samples from those whose mRNA expression had been altered by suboptimal handling.

In Figure 1, you can see a PCA plot showing two clusters corresponding to the four patient cohorts; however, some samples overlap, making the separation somewhat fuzzy. This is the PCA plot before EDTA QC filtering.


![Before-cleaning](./before.jpg)

In Figure 2, the same PCA plot is shown after EDTA QC flagged and removed suspect samples. The false negative rate was set to 5%. Under this condition, one-third of the samples (24/69) were excluded because they were impaired during EDTA tube handling.

![After cleaning](./after.jpg)

Conclusion
When the EDTA QC markers were used to clean this dataset:

    The cancer-marker panel achieved even higher specificity than originally reported (by removing potential bias introduced by EDTA tubes).

    It became clear why dedicated tubes and strict handling are essential for maintaining stable gene expression.

----
----

Go to application via terminal: streamlit run src/app.py